In [1]:
import requests
from bs4 import BeautifulSoup
import time

url = 'https://www.tripadvisor.com.tw/Hotels-g293913-Taipei-Hotels.html'
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}

def get_hotel(url):
    #每次调用等待两秒, 避免被ban ip
    time.sleep(2)
    
    uri_list = []
    
    resp = requests.get(url)

    soup = BeautifulSoup(resp.text,'html.parser')

    titles = soup.select('div.listing_title > a[target="_blank"]')

    data_list = []

    # 抓該頁各飯店資訊
    for i, title in enumerate(titles):
        # 放肯定會有的資料
        data = {
            'title':title.get_text(),
            'uri': 'https://www.tripadvisor.com.tw' + title.get('href')
        }
        
        uri_list.append(data['uri'])
        data_list.append(data)
    
    return uri_list, data_list

In [2]:
# 抓每頁30間飯店
all_uri = []
url_list = ['https://www.tripadvisor.com.tw/Hotels-g293913-oa{}-Taipei-Hotels.html'.format(str(i)) for i in range(0,1200,30)]
for k in range(0,10):
    hotel_url, hotels_data = get_hotel(url_list[k])
    all_uri = all_uri + hotel_url

In [65]:
def craw_nearby(url_list):
    data_list = []
    for i, url in enumerate(url_list):
        time.sleep(2)
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text,'html.parser')
        # 包含附近景點、交通、餐廳
        near_col = soup.select('.hotels-hotel-review-location-layout-LocationColumn__locationColumn--kANeS')
        new_layout = soup.select('.prw_rup.prw_common_btf_nearby_poi_grid.grid-widget')
        
        data = {'hotel_url': url}
        restaurant_list = []
        spot_list = []
        
        # 針對新的版面設計做處理
        if len(new_layout) > 2:
#             print(url)
            for j, restaurant in enumerate(new_layout[1].select('[data-prwidget-name="common_btf_nearby_poi_entry"]')):
                restaurant_name = restaurant.select('.poiName')
                if len(restaurant_name) > 0:
                    restaurant_list.append(restaurant_name[0].get_text())
                    
            for j, spot in enumerate(new_layout[2].select('[data-prwidget-name="common_btf_nearby_poi_entry"]')):
                spot_name = spot.select('.poiName')
                if len(spot_name) > 0:
                    spot_list.append(spot_name[0].get_text())
            data['restaurants_nearby'] = ', '.join(restaurant_list)
            data['spot_nearby'] = ', '.join(spot_list)
        
        elif len(near_col) > 2:
            try: 
                # 印附近前幾家餐廳(最多4間)
                for j, restaurant in enumerate(near_col[1].select('.hotels-hotel-review-location-NearbyLocation__name--1pAvV')):
                    restaurant_list.append(restaurant.get_text())
                for j, spot in enumerate(near_col[2].select('.hotels-hotel-review-location-NearbyLocation__name--1pAvV')):
                    spot_list.append(spot.get_text())
                data['restaurants_nearby'] = ', '.join(restaurant_list)
                data['spot_nearby'] = ', '.join(spot_list)

            except:
                # 沒有推薦附近餐廳的飯店網址(也有可能是js render超過10秒)
                print(url)
                continue
                
        data_list.append(data)
    return data_list

In [66]:
all_data = craw_nearby(all_uri)

In [67]:
import pandas as pd
# 用 list存多個 dict然後轉成 df
data_df = pd.DataFrame.from_dict(all_data)
data_df.head()

,hotel_url,restaurants_nearby,spot_nearby
0,https://www.tripadvisor.com.tw/Hotel_Review-g1...,"Prime One牛排館, hot 7 - 台北家樂福桂林店, 六國餐廳 台北花園大酒店, ...","家樂福(桂林店), 剝皮寮老街, 新富町文化市場, 艋舺清水巖祖師廟"
1,https://www.tripadvisor.com.tw/Hotel_Review-g1...,"藝奇 - 台北衡陽店, Mala Yuanyang Hotpot, 老牌牛肉拉麺大王, T....","總統府, 西門紅樓, 天后宮, 西門站地下商城"
2,https://www.tripadvisor.com.tw/Hotel_Review-g1...,"福州元祖胡椒餅, 周記肉粥, 四方阿九魯肉飯, 陳記腸蚵專業麵線","龍山寺, 華西街觀光夜市, 龍都冰果專業家, 廣州街觀光夜市"
3,https://www.tripadvisor.com.tw/Hotel_Review-g1...,,
4,https://www.tripadvisor.com.tw/Hotel_Review-g1...,,


In [69]:
data_df.to_csv('./tapei_tripadvisor_nearby.csv', index=False, encoding='utf_8_sig')